In [1]:
%load_ext lab_black

In [2]:
# !pip install google-search-results

In [4]:
import os

with open("../API_KEY", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read()
# with open("../SERPAPI_KEY", "r") as f:
#     os.environ["SERPAPI_API_KEY"] = f.read()

In [5]:
import langchain

In [6]:
# Import things that are needed generically
from langchain.utilities import PythonREPL

# from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

# Aims:

- Build a coding Agent
- Basically try to write code that meets user requirements from mostly natural language input (maybe some toggles/buttons etc in dash)
- Make tools for different types of coding tasks (e.g. High level class creation (defs and docstring only), Low level function code (based on defs/docstring), commenting, refining (commenting/type hinting/refactoring), summarizing)
- Keep track of messages during use (later can use them to generate graphs, good/bad examples, etc). Should be stored in vector database? (and normal database?)
- Could later add tool that searches readthedocs or stackeoverflow etc
- Store code in vector database (and normal database?)

In [99]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.chains import (
    ConversationalRetrievalChain,
    ConversationChain,
    LLMChain,
    SequentialChain,
)
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain.memory import ChatMessageHistory
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [100]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(
    """
Given the following conversation history: 

---------------
{chat_history}
---------------

and follow up question:

---------------
Follow Up Question: {question}
---------------

Briefly summarize the information in the conversation history that is relevant to the follow up question, then rephrase the follow up question to be a standalone question. You should format your response like:
---------------
Possibly Relevant info:
<relevant info here>

<standalone question here>
---------------
"""
)
condense_input_chain = LLMChain(
    llm=ChatOpenAI(temperature=0.0),
    prompt=CONDENSE_QUESTION_PROMPT,
    output_key="condensed_question",
    verbose=True,
)

In [101]:
chat_history = ChatMessageHistory(
    messages=[
        HumanMessage(content="How many legs does a cat have?"),
        AIMessage(content="A cat has 4 legs"),
        HumanMessage(content="What about a spider?"),
        AIMessage(content="A spider has 8 legs"),
    ]
)
chat_prompt_template = ChatPromptTemplate(
    input_variables=[], messages=chat_history.messages
)

In [102]:
print(chat_prompt_template.format())

Human: How many legs does a cat have?
AI: A cat has 4 legs
Human: What about a spider?
AI: A spider has 8 legs


In [103]:
print(
    condense_input_chain.prompt.format(
        question="What animals have I mentioned in the chat so far?",
        chat_history=chat_prompt_template.format(),
    )
)


Given the following conversation history: 

---------------
Human: How many legs does a cat have?
AI: A cat has 4 legs
Human: What about a spider?
AI: A spider has 8 legs
---------------

and follow up question:

---------------
Follow Up Question: What animals have I mentioned in the chat so far?
---------------

Briefly summarize the information in the conversation history that is relevant to the follow up question, then rephrase the follow up question to be a standalone question. You should format your response like:
---------------
Possibly Relevant info:
<relevant info here>

<standalone question here>
---------------



In [104]:
condensed_q = condense_input_chain.run(
    question="What animals have I mentioned in the chat so far?",
    chat_history=chat_prompt_template.format(),
)
print(condensed_q)



> Entering new LLMChain chain...
Prompt after formatting:

Given the following conversation history: 

---------------
Human: How many legs does a cat have?
AI: A cat has 4 legs
Human: What about a spider?
AI: A spider has 8 legs
---------------

and follow up question:

---------------
Follow Up Question: What animals have I mentioned in the chat so far?
---------------

Briefly summarize the information in the conversation history that is relevant to the follow up question, then rephrase the follow up question to be a standalone question. You should format your response like:
---------------
Possibly Relevant info:
<relevant info here>

<standalone question here>
---------------


> Finished chain.
Possibly Relevant info:
The human asked about the number of legs a cat and a spider have.

What animals have been mentioned in the chat so far?



In [106]:
question_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="""
You are a coding assistant that helps the user write and/or understand code. When you are answering you should follow these rules:
- If you don't know the answer, don't try to make up an answer, instead, tell the user that they should try rephrasing their question or include more context with their question.
- You should usually aim to give code answers or examples (usually using Python) unless the user is asking for an explanation.
- If it's appropriate to answer the users question using only code, then do that. Avoid giving text explanations unless asked.
- It is absolutely crucial that you give your responses formatted in Markdown, especially any part of the answer that contains code should be enclosed by triple back-ticks
- You should follow the SOLID coding principles when possible
"""
        ),
        HumanMessagePromptTemplate.from_template("{condensed_question}"),
    ],
)

generate_answer_chain = LLMChain(
    llm=ChatOpenAI(temperature=0.0), prompt=question_prompt_template, verbose=True
)

In [107]:
print(generate_answer_chain.prompt.format(condensed_question=condensed_q))

System: 
You are a coding assistant that helps the user write and/or understand code. When you are answering you should follow these rules:
- If you don't know the answer, don't try to make up an answer, instead, tell the user that they should try rephrasing their question or include more context with their question.
- You should usually aim to give code answers or examples (usually using Python) unless the user is asking for an explanation.
- If it's appropriate to answer the users question using only code, then do that. Avoid giving text explanations unless asked.
- It is absolutely crucial that you give your responses formatted in Markdown, especially any part of the answer that contains code should be enclosed by triple back-ticks
- You should follow the SOLID coding principles when possible

Human: Possibly Relevant info:
The human asked about the number of legs a cat and a spider have.

What animals have been mentioned in the chat so far?



In [108]:
full_chain = SequentialChain(
    input_variables=["question", "chat_history"],
    chains=[condense_input_chain, generate_answer_chain],
    verbose=True,
)

In [109]:
from IPython.display import Markdown

response = full_chain.run(
    question="How can I generate the first 30 numbers in the fibonacci sequence then save them to a text file?",
    chat_history="",
)
Markdown(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Given the following conversation history: 

---------------

---------------

and follow up question:

---------------
Follow Up Question: How can I generate the first 30 numbers in the fibonacci sequence then save them to a text file?
---------------

Briefly summarize the information in the conversation history that is relevant to the follow up question, then rephrase the follow up question to be a standalone question. You should format your response like:
---------------
Possibly Relevant info:
<relevant info here>

<standalone question here>
---------------


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a coding assistant that helps the user write and/or understand code. When you are answering you should follow these rules:
- If you don't know the answer, don't try to make up an answer, instead, tell the user that they should try r

You can generate the first 30 numbers in the Fibonacci sequence and save them to a text file using the following Python code:

```python
# Generate the first 30 numbers in the Fibonacci sequence
fibonacci = [0, 1]
for i in range(2, 30):
    fibonacci.append(fibonacci[i-1] + fibonacci[i-2])

# Save the numbers to a text file
with open('fibonacci.txt', 'w') as file:
    for number in fibonacci:
        file.write(str(number) + '\n')
```

This code first generates the first 30 numbers in the Fibonacci sequence using a loop and stores them in a list called `fibonacci`. Then, it opens a file called `fibonacci.txt` in write mode using a `with` statement, and writes each number in the list to the file, separated by a newline character.

In [ ]:
raise

In [5]:
llm = ChatOpenAI(temperature=0)

In [11]:
# Load the tool configs that are needed.
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool.from_function(
        func=search.run,
        name="Search",
        description="useful for when you need to answer questions about current events"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
]

In [12]:
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    question: str = Field()


tools.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput
        # coroutine= ... <- you can specify an async method if desired as well
    )
)

In [13]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [14]:
# agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")
agent.run(
    "What is the temperature today in Vancouver? What is that temperature raised to the power 2.75?"
)



> Entering new AgentExecutor chain...
I need to find out the current temperature in Vancouver and then raise it to the power of 2.75. 
Action: Search 
Action Input: "Vancouver temperature today" 
Observation: Current ConditionsObserved at: Vancouver Int'l Airport 11:00 AM PDT Sunday 21 May 2023 ; Condition: Mostly Cloudy; Pressure: 102.1 kPa ; Temperature: 15.6° C · Dew ...
Thought:I have the current temperature in Vancouver, now I need to raise it to the power of 2.75. 
Action: Calculator 
Action Input: 15.6^2.75 

> Entering new LLMMathChain chain...
15.6^2.75

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


```text
15.6**2.75
```
...numexpr.evaluate("15.6**2.75")...

Answer: 1910.2607197321036
> Finished chain.

Observation: Answer: 1910.2607197321036
Thought:I now know the final answer. 
Final Answer: The temperature in Vancouver today is 15.6°C and when raised to the power of 2.75, it is 1910.2607197321036.

> Finished chain.


'The temperature in Vancouver today is 15.6°C and when raised to the power of 2.75, it is 1910.2607197321036.'